In [1]:
import pyspark as ps
spark = (ps.sql.SparkSession.builder 
        .master("local[4]") 
        .appName("yelp_academic") 
        .getOrCreate()
        )
sc = spark.sparkContext

In [59]:
from pyspark.ml.recommendation import ALS
from pyspark.sql.types import *
from pyspark.sql.functions import udf

In [22]:
from pymongo import MongoClient

mc = MongoClient()
db  = mc['raw_restaurants']
bisn = db['restaurants']
users = db['users']
rv = db['reviews']
rv_s = db['reviews_scrap']

In [23]:
biz = spark.read.json('yelp_dataset/yelp_academic_dataset_business.json')

In [3]:
rev = spark.read.json('yelp_dataset/yelp_academic_dataset_review.json')

In [5]:
user = spark.read.json('yelp_dataset/yelp_academic_dataset_user.json')

<i> Next cell is not need for now </i>

In [24]:
biz.createOrReplaceTempView('business')
rev.createOrReplaceTempView('reviews')

In [27]:
biz.count(), rev.count()

(188593, 5996996)

In [40]:
cities = biz.select('city')
list_of_c = cities.groupBy('city').count().collect()

In [42]:
len(list_of_c)

1111

In [57]:
states = biz.select('state').groupBy('state').count().collect()

In [58]:
biz.select('categories').take(2)

[Row(categories='Tours, Breweries, Pizza, Restaurants, Food, Hotels & Travel'),
 Row(categories='Chicken Wings, Burgers, Caterers, Street Vendors, Barbeque, Food Trucks, Food, Restaurants, Event Planning & Services')]

In [83]:
from train_model import str_to_l, if_rest_udf, str_to_list_udf

In [62]:
str_to_l('Tours, Breweries, Pizza, Restaurants, Food, Hotels & Travel')

['Tours', 'Breweries', 'Pizza', 'Restaurants', 'Food', 'Hotels & Travel']

In [69]:
str_to_list_udf = udf(str_to_l, ArrayType(StringType()))

In [77]:
rests = biz.filter(if_rest_udf(biz.categories))

In [101]:
rests.cache()

DataFrame[address: string, attributes: struct<AcceptsInsurance:string,AgesAllowed:string,Alcohol:string,Ambience:string,BYOB:string,BYOBCorkage:string,BestNights:string,BikeParking:string,BusinessAcceptsBitcoin:string,BusinessAcceptsCreditCards:string,BusinessParking:string,ByAppointmentOnly:string,Caters:string,CoatCheck:string,Corkage:string,DietaryRestrictions:string,DogsAllowed:string,DriveThru:string,GoodForDancing:string,GoodForKids:string,GoodForMeal:string,HairSpecializesIn:string,HappyHour:string,HasTV:string,Music:string,NoiseLevel:string,Open24Hours:string,OutdoorSeating:string,RestaurantsAttire:string,RestaurantsCounterService:string,RestaurantsDelivery:string,RestaurantsGoodForGroups:string,RestaurantsPriceRange2:string,RestaurantsReservations:string,RestaurantsTableService:string,RestaurantsTakeOut:string,Smoking:string,WheelchairAccessible:string,WiFi:string>, business_id: string, categories: string, city: string, hours: struct<Friday:string,Monday:string,Saturday:string

In [79]:
#rests = rests.withColumn('categories', str_to_list_udf(rests.categories))

In [93]:
##Wanna try two ways
rests.createOrReplaceTempView('rests')
rev.createOrReplaceTempView('reviews')

In [91]:
rest_id = rests.select('business_id')

In [97]:
rest_id.createOrReplaceTempView('rests_id')

In [98]:
%%time
select = '''
SELECT  reviews.* 
FROM rests_id 
LEFT JOIN reviews ON rests_id.business_id == reviews.business_id


'''
rest_rev = spark.sql(select)
rest_rev.show(5)

+--------------------+----+----------+-----+--------------------+-----+--------------------+------+--------------------+
|         business_id|cool|      date|funny|           review_id|stars|                text|useful|             user_id|
+--------------------+----+----------+-----+--------------------+-----+--------------------+------+--------------------+
|--9e1ONYQuAa-CB_R...|   0|2014-02-25|    0|8eEp26oqTXmuIjVUJ...|    3|It was a real up ...|     0|AdxhlxXSGjRsCQxG6...|
|--9e1ONYQuAa-CB_R...|   0|2017-02-14|    0|VETXTwMw6qxzOVDlX...|    5|went for dinner t...|     0|ymlnR8UeFvB4FZL56...|
|--9e1ONYQuAa-CB_R...|   0|2013-12-19|    0|NQt645IJwa5ADM26U...|    5|we are a repeat c...|     0|TMqFvYbWqs8BnjLsE...|
|--9e1ONYQuAa-CB_R...|   0|2013-12-02|    2|G9TSEZ8BfuSmcf5xy...|    3|Thanksgiving dinn...|     5|KmubWHd-t2d3dxLZ1...|
|--9e1ONYQuAa-CB_R...|   0|2015-01-01|    0|6prMgAjp2LZjuxRvm...|    4|We were seated pr...|     0|bMFqD4beGrt_LjNGG...|
+--------------------+----+-----

In [96]:
%%time
rest_rev2 = rest_id.join(rev, "business_id")
rest_rev2.show(5)

+--------------------+----+----------+-----+--------------------+-----+--------------------+------+--------------------+
|         business_id|cool|      date|funny|           review_id|stars|                text|useful|             user_id|
+--------------------+----+----------+-----+--------------------+-----+--------------------+------+--------------------+
|iCQpiavjjPzJ5_3gP...|   0|2011-02-25|    0|x7mDIiDB3jEiPGPHO...|    2|The pizza was oka...|     0|msQe1u7Z_XuqjGoqh...|
|pomGBqfbxcqPv14c3...|   0|2012-11-13|    0|dDl8zu1vWPdKGihJr...|    5|I love this place...|     0|msQe1u7Z_XuqjGoqh...|
|jtQARsP6P-LbkyjbO...|   1|2014-10-23|    1|LZp4UX5zK3e-c5ZGS...|    1|Terrible. Dry cor...|     3|msQe1u7Z_XuqjGoqh...|
|elqbBhBfElMNSrjFq...|   0|2011-02-25|    0|Er4NBWCmCD4nM8_p1...|    2|Back in 2005-2007...|     2|msQe1u7Z_XuqjGoqh...|
|Ums3gaP2qM3W1XcA5...|   0|2014-09-05|    0|jsDu6QEJHbwP2Blom...|    5|Delicious healthy...|     0|msQe1u7Z_XuqjGoqh...|
+--------------------+----+-----

### Join faster??? after lunch try in diff order

In [99]:
rest_rev2.count()

3654797

In [102]:
rest_rev2.cache()

DataFrame[business_id: string, cool: bigint, date: string, funny: bigint, review_id: string, stars: bigint, text: string, useful: bigint, user_id: string]

In [ ]:
Calculate 